In [ ]:
import os
import json

from promptflow.core import AzureOpenAIModelConfiguration

# Initialize Azure OpenAI Connection
model_config = AzureOpenAIModelConfiguration(
        azure_deployment=os.environ["AZURE_DEPLOYMENT_NAME"],
        api_key=os.environ["AZURE_OPENAI_API_KEY"],
        api_version=os.environ["AZURE_OPENAI_API_VERSION"],
        azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"]
    )

In [ ]:
import pandas as pd

data_path = "../data/data.jsonl"

df = pd.read_json(data_path, lines=True)
df.head()

In [ ]:
from promptflow.client import load_flow

# load apology evaluatorfrom prompty
groundedness_eval = load_flow(source="prompty/groundedness.prompty", model={"configuration": model_config})
fluency_eval = load_flow(source="prompty/fluency.prompty", model={"configuration": model_config})
coherence_eval = load_flow(source="prompty/coherence.prompty", model={"configuration": model_config})
relevance_eval = load_flow(source="prompty/relevance.prompty", model={"configuration": model_config})

In [ ]:
import sys
sys.path.append('../contoso_chat')  # Replace '/path/to/contoso_chat' with the actual path to the 'contoso_chat' folder

from chat_request import get_response


In [ ]:

results = []

for index, row in df.iterrows():
    customerId = row['customerId']
    question = row['question']
    
    # Run contoso-chat/chat_request flow to get response
    response = get_response(customerId=customerId, question=question, chat_history=[])
    
    # Add results to list
    result = {
        'customerId': customerId,
        'question': question,
        'response': response
    }
    results.append(result)

# Save results to a JSONL file
with open('result.jsonl', 'w') as file:
    for result in results:
        file.write(json.dumps(result) + '\n')

In [ ]:
from promptflow.evals.evaluate import evaluate
#from promptflow.evals.evaluators.content_safty import violence_eval

result_eval = evaluate(
    data="result.jsonl",
    evaluators={
        #"violence": violence_eval,
        "groundedness": groundedness_eval,
        "fluency": fluency_eval,
        "coherence": coherence_eval,
        "relevence": relevance_eval,
    },
    # column mapping
    evaluator_config={

            "question": "${response.question}",
            "answer": "${response.answer}",
            "context": "${response.context}",

    }
)

In [ ]:
eval_result = pd.DataFrame(result_eval["rows"])

In [ ]:
eval_result.head()